In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
!pip install fastai2>=0.0.11


In [ ]:
#inspiring from
#https://www.analyticsvidhya.com/blog/2021/05/training-state-of-the-art-deep-learning-models-with-fast-ai/
#https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb
#https://davideclark314.medium.com/learning-deep-learning-mnist-with-fastai-part-2-e81c281f063c
import numpy as np # linear algebra  #OUI
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# import fast ai vision library
from fastai.vision.all import *
from pathlib import Path
data_path = Path("../input/bird-speciestiny")
df = pd.read_csv(data_path/'bird-species-tiny.csv')
df.tail()

In [ ]:
dls = ImageDataLoaders.from_folder(path=data_path, 
                                   train='train', 
                                   valid='valid',
                                   shuffle=True)
dls.train.show_batch()
dls.valid.show_batch()

In [ ]:
dls_test = ImageDataLoaders.from_folder(path=data_path, 
                                   train='train', 
                                   valid='test',
                                   shuffle=True)
dls_test.train.show_batch()
dls_test.valid.show_batch()

In [ ]:
learn = cnn_learner(dls, 
                    resnet50, 
                    metrics=[accuracy, error_rate], model_dir="/kaggle/working")


#learn.fine_tune(4)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit(4, lr=1e-3)

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.save("model")
learn.fit_one_cycle(16, lr_max=slice(1e-6,1e-5), cbs=[SaveModelCallback(monitor="accuracy")])


In [ ]:
learn.load("model")

In [ ]:
# get all the image paths from testing folder
data_path = Path("../input/bird-speciestiny")
test_images = get_image_files(data_path/"test")
len(test_images)

In [ ]:
preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))

In [ ]:
imglabels = list(df.columns[3:])
imglabels 

In [ ]:
# ensure that the order of columns in preds matches the imglabels
#preds = preds[0].cpu().numpy()
#vocab = list(dls[0].dataset.vocab)
#column_permutation = [vocab.index(l) for l in imglabels]
#preds = preds[:, column_permutation]

submission = pd.DataFrame()
#submission["image_id"] = test_image_ids
for i in range(len(imglabels)):
    submission[imglabels[i]] = preds[:, i]
submission.to_csv("submission.csv", index=False)

In [ ]:
submission.head(10)

In [ ]:
# get all the image paths from testing folder
data_path = Path("../input/bird-speciestiny")
images = get_image_files(data_path/"test")
len(images)
# select an image and display
img = PILImage.create(images[4])
img


In [ ]:
# predict the image class
lbl, _ , _ = learn.predict(img)
lbl

In [ ]:
interep = ClassificationInterpretation.from_learner(learn)
interep.plot_confusion_matrix()

In [ ]:
interep.plot_top_losses(10, figsize=(35,15)) #click to enlarge